In [ ]:
import torch
import pandas as pd
import numpy as np

In [ ]:

def run_length_decode(rle, height=1024, width=1024, fill_value=1):

    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)

    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end

    component = component.reshape(width, height).T
    return component



In [ ]:

def metric(probability, truth, threshold=0.5, reduction='none'):
    '''probability and truth must be torch tensors'''
    batch_size = len(truth)

    with torch.no_grad():

        probability = probability.view(batch_size, -1)
        truth = truth.view(batch_size, -1)
        assert(probability.shape == truth.shape)

        p = (probability > threshold).float()
        t = (truth > 0.5).float()

        t_sum = t.sum(-1)
        p_sum = p.sum(-1)
        neg_index = torch.nonzero(t_sum == 0)
        pos_index = torch.nonzero(t_sum >= 1)

        dice_neg = (p_sum == 0).float()
        dice_pos = 2 * (p*t).sum(-1)/((p+t).sum(-1))

        dice_neg = dice_neg[neg_index]
        dice_pos = dice_pos[pos_index]
        dice = torch.cat([dice_pos, dice_neg])

        dice_neg = np.nan_to_num(dice_neg.mean().item(), 0)
        dice_pos = np.nan_to_num(dice_pos.mean().item(), 0)
        dice = dice.mean().item()

        num_neg = len(neg_index)
        num_pos = len(pos_index)

    return dice, dice_neg, dice_pos, num_neg, num_pos



In [ ]:
train2 = pd.read_csv('../input/stage_2_train.csv')
test1 = pd.read_csv('../input/test_model.csv')
#test1['preds']
train2['isTest'] = train2.apply(lambda x: 1 if x[0] in test1.ImageId.tolist() else 0, axis=1) # bool for whether the image belongs to stage1 test set or not

train2['isTest'].value_counts()

train2 = train2.query('isTest == 1') # get the test images out

train2.query('EncodedPixels == "-1"').shape,  test1.query('EncodedPixels == "-1"').shape, 

train2.head()

test1 = test1.rename(columns={'EncodedPixels': 'preds'}) # renaming

df = pd.merge(train2, test1, on="ImageId") # merge ground truth and predictions in one dataframe

df.head() # EncodedPixels is ground truth, preds is predicted mask


In [ ]:

def get_mask(rle):
    if rle!='-1':
        mask = run_length_decode(rle)
        return mask
    else:
        return np.zeros([1024, 1024], dtype='uint8')

mask = get_mask('-1')
mask.shape, mask.dtype

labels = []
predictions = []
for itr, row in df.iterrows():
    gt = row['EncodedPixels']
    pred = row['preds']
    labels.append(get_mask(gt))
    predictions.append(get_mask(pred))

labels = np.array(labels)
predictions = np.array(predictions)
labels.shape, predictions.shape

labels = torch.from_numpy(labels)
predictions = torch.from_numpy(predictions)


In [ ]:
dice, dice_neg, dice_pos, num_neg, num_pos = metric(predictions, labels, threshold=0.5, reduction='none')